Tutorial link : https://towardsdatascience.com/build-your-first-cnn-with-tensorflow-a9d7394eaa2e

Dataset link : https://www.kaggle.com/datasets/tanlikesmath/the-oxfordiiit-pet-dataset

In [ ]:
!pip install tensorflow
!pip install numpy
!pip install pandas

In [ ]:
import pandas as pd
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Get data

 We will create two lists, one with all the photos of dogs and the other with all the images of cats.

In [ ]:
CATS = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair', 'Egyptian_Mau', 'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue', 'Siamese', 'Sphynx']

cats_images = []
dogs_images = []

for img in glob.glob('images/*.jpg'):
    if any(cat in img for cat in CATS):
        cats_images.append(img)
    else:
        dogs_images.append(img)

In [ ]:
#shuffle data
np.random.shuffle(cats_images)
np.random.shuffle(dogs_images)

In [ ]:
#divide into train, validation and test sets
train_d, val_d, test_d = np.split(dogs_images, [int(len(dogs_images)*0.7), int(len(dogs_images)*0.8)])
train_c, val_c, test_c = np.split(cats_images, [int(len(cats_images)*0.7), int(len(cats_images)*0.8)])

In [ ]:
# Now create dataframes using pandas
# 2 cols, onw with image name, another with label "cat" or "dog"


train_dog_df = pd.DataFrame({'image':train_d, 'label':'dog'})
val_dog_df = pd.DataFrame({'image':val_d, 'label':'dog'})
test_dog_df = pd.DataFrame({'image':test_d, 'label':'dog'})

train_cat_df = pd.DataFrame({'image':train_c, 'label':'cat'})
val_cat_df = pd.DataFrame({'image':val_c, 'label':'cat'})
test_cat_df = pd.DataFrame({'image':test_c, 'label':'cat'})

In [ ]:
#now concat thes dataframes, and create trainj, val and test data

train_df = pd.concat([train_dog_df, train_cat_df])
val_df = pd.concat([val_dog_df, val_cat_df])
test_df = pd.concat([test_dog_df, test_cat_df])

# Preprocess data

Should have same dimensions, normalize pixels, create batches

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = IMG_WIDTH = 224

#create the ImageDataGenerator object and rescale the images
train_gen = ImageDataGenerator(rescale = 1./255)
val_gen = ImageDataGenerator(rescale = 1./255)
test_gen = ImageDataGenerator(rescale = 1./255)

#convert them into a dataset
train_dataset = train_gen.flow_from_dataframe(
    dataframe = train_df,
    class_mode = "binary",
    x_col = "image",
    y_col = "label",
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    target_size = (IMG_HEIGHT, IMG_WIDTH)
)

val_dataset = val_gen.flow_from_dataframe(
    dataframe = val_df,
    class_mode = "binary",
    x_col = "image",
    y_col = "label",
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    target_size = (IMG_HEIGHT, IMG_WIDTH)
)

test_dataset = test_gen.flow_from_dataframe(
    dataframe = test_df,
    class_mode = "binary",
    x_col = "image",
    y_col = "label",
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    target_size = (IMG_HEIGHT, IMG_WIDTH)
)


# Create model

In [ ]:
model = tf.keras.Sequential([
    keras.layers.InputLayer(input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(256, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(512, (3, 3), activation='relu'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [ ]:
#for PIL error
!pip install pillow                     
!pip install scikit-learn

In [ ]:
model.fit(
    train_dataset,
    epochs = 15,
    validation_data = (val_dataset)
)